# Lab 1: Microsoft Agent Framework Basics

**Duration:** 15 minutes  
**Objective:** Understand and implement the core agent primitives: Agent, Thread, Run, and Tools

---

## What is the Microsoft Agent Framework?

> [Microsoft Agent Framework](https://github.com/microsoft/agent-framework) is an open-source development kit for building **AI agents** and **multi-agent workflows** for .NET and Python. It brings together and extends ideas from [Semantic Kernel](https://github.com/microsoft/semantic-kernel) and [AutoGen](https://github.com/microsoft/autogen) projects, combining their strengths while adding new capabilities.
>
> — [Agent Framework Overview](https://learn.microsoft.com/agent-framework/overview/agent-framework-overview)

### Framework Capabilities

The framework offers two primary categories of capabilities:

| Category | Description |
|----------|-------------|
| **AI Agents** | Individual agents that use LLMs to process user inputs, call tools and MCP servers to perform actions, and generate responses |
| **Workflows** | Graph-based workflows that connect multiple agents and functions to perform complex, multi-step tasks |

### When to Use AI Agents

AI agents excel in scenarios requiring:
- **Autonomous decision-making** and ad hoc planning
- **Trial-and-error exploration** and conversation-based interactions
- **Customer Support** - handling multi-modal queries using tools to look up information
- **Code Generation** - assisting with implementation, reviews, and debugging
- **Research Assistance** - searching, summarizing, and synthesizing information

> *If you can write a function to handle the task, do that instead of using an AI agent. You can use AI to help you write that function.*

---

## Prerequisites

- Azure subscription with AI Foundry access
- Python 3.10+
- An AI Foundry project with a deployed model (GPT-4o recommended)
- **Azure RBAC**: Your identity needs "Azure AI Developer" or "Contributor" role on the project
- **Environment Setup**: Copy `.env.sample` to `.env` and set your `PROJECT_ENDPOINT`

---

## Step 1: Environment Setup

Install the required SDK and configure your connection to Azure AI Foundry using project endpoints and Azure RBAC.

### Installation

```bash
pip install agent-framework --pre
```

The Agent Framework provides foundational building blocks including:
- **Model clients** (chat completions and responses)
- **Agent session** for state management  
- **Context providers** for agent memory
- **Middleware** for intercepting agent actions
- **MCP clients** for tool integration

> Learn more: [Agent Framework Installation](https://learn.microsoft.com/agent-framework/overview/agent-framework-overview#installation)

In [ ]:
# Install dependencies (--pre required for preview SDK)
%pip install agent-framework --pre python-dotenv --quiet

In [ ]:
import os
from typing import Annotated
from pydantic import Field
from dotenv import load_dotenv

from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import DefaultAzureCredential

# Load environment variables from .env file
# Copy .env.sample to .env and fill in your values
load_dotenv()

# Configuration - loaded from .env file
# Use the Project Endpoint URL (found in Azure AI Foundry portal under Project Settings)
PROJECT_ENDPOINT = os.getenv("PROJECT_ENDPOINT")

if not PROJECT_ENDPOINT or PROJECT_ENDPOINT.startswith("https://<"):
    raise ValueError("Please set PROJECT_ENDPOINT in your .env file (copy from .env.sample)")

# Initialize credential for Azure RBAC authentication
# Ensure your identity has "Azure AI Developer" or "Contributor" role on the project
credential = DefaultAzureCredential()

print("✓ Agent Framework SDK loaded")
print("✓ Environment loaded from .env")
print("✓ Azure credential initialized (using RBAC)")
print(f"✓ Project endpoint: {PROJECT_ENDPOINT}")

---

## Step 2: Define a Custom Tool (Function)

Before creating the agent, let's define a custom tool. This demonstrates how agents can invoke external functions.

### What are Tools?

> Tools are Python functions that the agent can call to perform specific tasks like retrieving data, performing calculations, or interacting with external systems.
>
> — [Agent Framework Tools](https://learn.microsoft.com/agent-framework/user-guide/agents/agent-tools)

### Creating Function Tools

With Agent Framework, tools are defined using:
- **`@tool` decorator** or just passing a function directly
- **Type annotations** with `Annotated` and `Field` for parameter descriptions
- **Docstrings** that describe what the function does (helps the agent decide when to use it)

```python
@tool
def get_weather(
    location: Annotated[str, Field(description="The city")],
) -> str:
    """Get the current weather for a location."""
    return f"The weather in {location} is sunny."
```

**Scenario:** A simple "get_system_metrics" tool that simulates fetching infrastructure metrics.

In [ ]:
# Define a tool using type hints - the SDK automatically generates the schema
# No manual JSON schema required!

def get_system_metrics(
    server_name: Annotated[str, Field(description="The name of the server (e.g., vm-prod-01, vm-db-01)")],
    metric_type: Annotated[str, Field(description="The type of metric: cpu, memory, disk, network_in, or all")]
) -> dict:
    """
    Retrieves current system metrics (CPU, memory, disk, network) for a specified server.
    Use this to diagnose performance issues.
    """
    # Simulated metrics data
    metrics_data = {
        "vm-prod-01": {"cpu": 78.5, "memory": 62.3, "disk": 45.0, "network_in": 125.6},
        "vm-prod-02": {"cpu": 23.1, "memory": 41.2, "disk": 72.8, "network_in": 89.3},
        "vm-db-01": {"cpu": 91.2, "memory": 88.5, "disk": 55.0, "network_in": 234.1},
    }
    
    server = metrics_data.get(server_name, {})
    if not server:
        return {"error": f"Server {server_name} not found"}
    
    if metric_type == "all":
        return {"server": server_name, "metrics": server}
    
    value = server.get(metric_type)
    if value is None:
        return {"error": f"Metric {metric_type} not found for {server_name}"}
    
    return {"server": server_name, "metric": metric_type, "value": value, "unit": "%" if metric_type != "network_in" else "Mbps"}

# Test the function locally
print(get_system_metrics("vm-prod-01", "cpu"))
print(get_system_metrics("vm-db-01", "all"))

In [ ]:
# With Agent Framework, tools are just Python functions!
# The SDK automatically:
# - Extracts the function name
# - Uses the docstring as the description  
# - Generates JSON schema from type hints and Field() descriptions
# - Handles tool invocation and response parsing

# No manual FunctionTool schema required - just pass the function to the agent

print("✓ Tool defined: get_system_metrics")
print("  The SDK will auto-generate the schema from type hints")

---

## Step 3: Create the Agent

### What is an AI Agent?

> An **AI agent** uses an LLM to process user inputs, make decisions, call tools and MCP servers to perform actions, and generate responses.
>
> — [Agent Framework Overview](https://learn.microsoft.com/agent-framework/overview/agent-framework-overview#what-is-an-ai-agent)

### Agent Components

![AI Agent Diagram](https://learn.microsoft.com/agent-framework/media/agent.svg)

An **Agent** is the AI persona with:
- **Instructions**: System prompt defining behavior
- **Model**: The LLM to use (from your AI Foundry deployment)
- **Tools**: Capabilities the agent can invoke

### Additional Capabilities

An AI agent can also be augmented with:
- **Session** - for state management across runs
- **Context providers** - for agent memory (long-term and short-term)
- **Middleware** - for intercepting and filtering agent actions

The `AzureAIAgentClient` uses your project endpoint and Azure RBAC credentials.

> Learn more: [Creating Agents](https://learn.microsoft.com/agent-framework/tutorials/agents/run-agent)

In [ ]:
# Create the agent with ChatAgent and AzureAIAgentClient
# The client connects to Azure AI Foundry using your project endpoint + RBAC

chat_client = AzureAIAgentClient(
    project_endpoint=PROJECT_ENDPOINT,
    credential=credential,
    model_deployment_name="gpt-5.1"
)

agent = ChatAgent(
    chat_client=chat_client,
    name="SRE-Assistant",
    instructions="""
You are an expert Site Reliability Engineer assistant. Your role is to help diagnose 
and troubleshoot infrastructure issues.

When analyzing server issues:
1. First gather relevant metrics using the get_system_metrics tool
2. Analyze the data for anomalies (CPU > 80%, Memory > 85%, Disk > 90% are concerning)
3. Provide clear, actionable recommendations

Be concise but thorough. Format output with clear sections.
""",
    tools=[get_system_metrics],  # Just pass the function - SDK handles the rest!
)

print(f"✓ Agent created: {agent.name}")
print(f"  Tools: [get_system_metrics]")

---

## Step 4: Create a Thread (Multi-Turn Conversations)

### What is an AgentThread?

> The Microsoft Agent Framework provides built-in support for managing **multi-turn conversations** with AI agents. This includes maintaining context across multiple interactions. The `AgentThread` type is the abstraction that represents chat history and other state of an agent.
>
> — [Multi-Turn Conversations](https://learn.microsoft.com/agent-framework/user-guide/agents/multi-turn-conversation)

### Thread Capabilities

An `AgentThread` contains the state of a specific conversation which may include:

| State Type | Description |
|------------|-------------|
| **Conversation history** | Messages or a reference to externally stored history |
| **Memories** | Or a reference to externally stored memories |
| **Custom state** | Any other state the agent needs to persist |

### Thread Creation Options

```python
# Option 1: Create a new thread from the agent
thread = agent.get_new_thread()

# Option 2: Run without thread (creates temporary thread)
response = await agent.run("Hello")  # Throwaway thread
```

A **Thread** is a conversation container that:
- Persists message history (in-memory or server-side)
- Can be serialized, stored, and resumed at any time
- Maintains context across multiple runs

> Learn more: [Multi-Turn Conversations](https://learn.microsoft.com/agent-framework/user-guide/agents/multi-turn-conversation)

In [ ]:
# Get a new thread from the agent
# Threads maintain conversation history across multiple runs

thread = agent.get_new_thread()

print(f"✓ Thread created")
print(f"  Thread will persist conversation context across runs")

---

## Step 5: Add a Message and Create a Run

A **Run** is a single execution of an agent on a thread. When you create a run:
1. The agent processes all messages in the thread
2. May invoke tools as needed
3. Appends its response to the thread

In [ ]:
# With Agent Framework, sending a message and running the agent is a single call!
# No need to manually:
# - Add messages to thread
# - Create a run
# - Poll for status
# - Handle tool calls
# - Submit tool outputs
# The SDK handles all of this automatically

user_query = "I'm seeing alerts for vm-db-01. Can you check the metrics and tell me what's going on?"

print(f"User: {user_query}")
print(f"\nRunning agent (tools will be invoked automatically)...")

In [ ]:
# Run the agent - this single call does everything!
# - Sends the message
# - Processes with the LLM
# - Automatically invokes tools when needed
# - Returns the final response

result = await agent.run(user_query, thread=thread)

print("=" * 60)
print("AGENT RESPONSE")
print("=" * 60)
print(result.text)

In [ ]:
# Inspect the full response object
# AgentRunResponse contains:
# - text: The final text response
# - messages: All messages in the conversation
# - tool_calls: Any tool invocations made

print(f"Response text length: {len(result.text)} characters")
print(f"Messages in response: {len(result.messages)}")

# Show if tools were called
if hasattr(result, 'tool_calls') and result.tool_calls:
    print(f"\nTools invoked during this run:")
    for tc in result.tool_calls:
        print(f"  → {tc}")
else:
    print("\n(Tool call details available in agent traces)")

---

## Step 6: Retrieve the Response

After the run completes, the agent's response is appended to the thread as a new message.

In [ ]:
# Access conversation history from the response messages
print("=" * 60)
print("CONVERSATION THREAD")
print("=" * 60)

for msg in result.messages:
    role = str(getattr(msg, "role", "MESSAGE")).upper()
    content = msg.text if hasattr(msg, 'text') else str(msg)
    
    print(f"\n[{role}]")
    print("-" * 40)
    print(content[:500] + "..." if len(content) > 500 else content)

print("\n" + "=" * 60)

---

## Step 7: Continue the Conversation

The thread persists - we can add more messages and create new runs to continue the conversation with full context.

In [ ]:
# Continue the conversation - thread maintains full context
# The agent remembers the previous exchange about vm-db-01

follow_up = "Can you compare vm-db-01 to vm-prod-01 and vm-prod-02? Which servers need attention?"

print(f"User: {follow_up}")
print("\nRunning agent with conversation context...")

result2 = await agent.run(follow_up, thread=thread)

print("\n" + "=" * 60)
print("AGENT RESPONSE")
print("=" * 60)
print(result2.text)

---

## Step 8: Memory and State Persistence

Agent Framework provides sophisticated memory capabilities for maintaining context and state.

### Memory Types Overview

> Agent chat history and memory are crucial capabilities that allow agents to maintain context across conversations, remember user preferences, and provide personalized experiences.
>
> — [Agent Chat History and Memory](https://learn.microsoft.com/agent-framework/user-guide/agents/agent-memory)

| Memory Type | Description | Use Case |
|-------------|-------------|----------|
| **In-Memory Storage** | Default - conversation stored in `AgentThread` object | Development, short sessions |
| **In-Service Storage** | History stored in Azure AI Foundry | Persistent agents, production |
| **Third-Party Storage** | Custom `ChatMessageStore` implementations | Redis, databases, custom stores |
| **Context Providers** | Dynamic memory injection before each invocation | User preferences, RAG, external memory services |

### Thread Serialization for Persistence

Threads can be serialized and stored for later use, enabling conversations to survive application restarts:

```python
import json

# Serialize thread state
serialized_thread = await thread.serialize()

# Save to file/database
with open("thread_state.json", "w") as f:
    json.dump(serialized_thread, f)

# Later, restore the thread
with open("thread_state.json", "r") as f:
    thread_data = json.load(f)

restored_thread = await agent.deserialize_thread(thread_data)
```

> Learn more: [Persisting and Resuming Conversations](https://learn.microsoft.com/agent-framework/tutorials/agents/persisted-conversation)

In [ ]:
# Bonus: Streaming responses
# Use run_stream() for real-time token streaming

print("Streaming example:")
print("-" * 40)

async for chunk in agent.run_stream("Give me a brief summary of the server status.", thread=thread):
    print(chunk.text, end="", flush=True)

print("\n" + "-" * 40)
print("✓ Stream complete")

# Demonstrate thread serialization for persistence
# This enables conversations to survive application restarts

import json

# Serialize the current thread state
serialized_thread = await thread.serialize()

print("\n" + "=" * 60)
print("Thread Serialization Example")
print("=" * 60)
print(f"Serialized thread type: {type(serialized_thread)}")

# In production, save to database:
# await db.save("user_session_123", json.dumps(serialized_thread))

# Later, restore and continue conversation:
# thread_data = await db.load("user_session_123")
# restored_thread = await agent.deserialize_thread(thread_data)

print("\n✓ Thread can be serialized and restored for persistent conversations")

---

## Summary: Agent Framework Primitives

| Primitive | Purpose | SDK Class |
|-----------|---------|-----------|
| **ChatAgent** | AI persona with instructions, tools, and chat client | `agent_framework.ChatAgent` |
| **Thread** | Conversation state container with message history | `agent.get_new_thread()` |
| **Run** | Single execution via `agent.run()` - handles everything | Returns `AgentRunResponse` |
| **Tool** | Python function with type hints - auto-schema generation | Just a `def` with `Annotated` types |

### Key Simplifications in Agent Framework SDK

| Before (manual approach) | After (Agent Framework) |
|--------------------------|-------------------------|
| Manual FunctionTool JSON schema | Type hints + docstrings |
| Create run → poll → handle tools → poll | `await agent.run(message)` |
| Manual message creation | Pass message to `run()` |
| Tool output submission | Automatic |

### Memory & State Management

> The `AgentThread` type is the abstraction that represents chat history and other state of an agent. `AIAgent` instances are **stateless** and the same agent instance can be used with multiple `AgentThread` instances.
>
> — [Agent Framework Documentation](https://learn.microsoft.com/agent-framework/user-guide/agents/multi-turn-conversation)

| Storage Type | When to Use |
|--------------|-------------|
| **In-Memory** | Development, testing, short sessions |
| **Service-Stored** | Production with Azure AI Foundry Agents |
| **Custom Store** | Redis, databases, or specialized storage |
| **Context Providers** | Dynamic memory injection (Mem0, RAG, etc.) |

---

## Next Steps

Proceed to **Lab 2** to see how Azure SRE Agent applies these primitives at scale for operational intelligence.

Or explore **Lab: AG-UI Integration** to learn how to build web-based AI agent applications with real-time streaming and interactive UI components.

---

## References

| Resource | URL |
|----------|-----|
| Microsoft Agent Framework GitHub | https://github.com/microsoft/agent-framework |
| Agent Framework Python Samples | https://github.com/microsoft/agent-framework/tree/main/python/samples |
| **Agent Framework Overview** | https://learn.microsoft.com/agent-framework/overview/agent-framework-overview |
| Multi-Turn Conversations | https://learn.microsoft.com/agent-framework/user-guide/agents/multi-turn-conversation |
| Agent Memory & Chat History | https://learn.microsoft.com/agent-framework/user-guide/agents/agent-memory |
| Persisting Conversations | https://learn.microsoft.com/agent-framework/tutorials/agents/persisted-conversation |
| Agent Framework Quick Start | https://learn.microsoft.com/agent-framework/tutorials/quick-start |
| Introducing Agent Framework (Blog) | https://devblogs.microsoft.com/foundry/introducing-microsoft-agent-framework-the-open-source-engine-for-agentic-ai-apps/ |
| Semantic Kernel + Agent Framework | https://devblogs.microsoft.com/semantic-kernel/semantic-kernel-and-microsoft-agent-framework/ |
| Azure AI Foundry | https://azure.microsoft.com/products/ai-foundry |
| PyPI Package | https://pypi.org/project/agent-framework/ |